## **Assignment 4 - Detecting steric overlap**
### **Hugo Manuel Alves Henriques e Silva, hugoalv@student.chalmers.se**

#### **How to run:**
`Run the cells below in order. Make sure to have the datasets in the same folder as the notebook.`

### **Data**

In [16]:
def read_file(filename):
    atoms = {}
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                x = float(line[30:38].strip())
                y = float(line[38:46].strip())
                z = float(line[46:54].strip())
                atom_coords = (x, y, z)
                atom_number = (int(line[6:11].strip()))
                amino_acid_name = line[17:20].strip()
                atom_name = line[12:16].strip()
                molecule_number = line[22:28].strip()
                amino_acid_description = str(atom_number) + " " + str(amino_acid_name) + "   " + str(molecule_number) + "  " + str(atom_name)
                atoms[atom_number] = (amino_acid_description, atom_coords)
    return atoms

### **First Method:**

Simply run through all the atoms of one of the files and for each of them find at least one atom from the other file that is closer than the 

distance threshold. If such an atom is found, the two atoms overlap. inefficient, but simple.

In [17]:


def calculate_distance_between_2_points(point1, point2):
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2 + (point1[2] - point2[2])**2)**0.5


def check_overlaps(atoms1, atoms2, threshold=4.0):
    overlaps = []
    num_comp = 0
    for atom1 in atoms1:
        for atom2 in atoms2:
            num_comp += 1
            if calculate_distance_between_2_points(atoms1[atom1][1], atoms2[atom2][1]) < threshold:
                overlap = str(atoms1[atom1][0])
                overlaps.append(overlap)
                break
    return overlaps, num_comp

atoms2 = read_file('1cdh.pdb')
atoms1 = read_file('2csn.pdb')

overlaps, num_comp = check_overlaps(atoms1, atoms2)

print("Number of comparisons: ", num_comp)
print("Number of overlaps: ", len(overlaps))
print("Overlaps:")
with open('overlaps.txt', 'w') as file:
    file.write("Number of comparisons: " + str(num_comp) + '\n')
    file.write("Number of overlaps: " + str(len(overlaps)) + '\n')
    for overlap in overlaps:
        file.write(overlap + '\n')
        print(overlap)


Number of comparisons:  3368296
Number of overlaps:  850
Overlaps:
674 SER   91  C
675 SER   91  O
678 LEU   92  N
679 LEU   92  CA
680 LEU   92  C
681 LEU   92  O
682 LEU   92  CB
683 LEU   92  CG
684 LEU   92  CD1
685 LEU   92  CD2
686 GLU   93  N
687 GLU   93  CA
688 GLU   93  C
689 GLU   93  O
690 GLU   93  CB
691 GLU   93  CG
692 GLU   93  CD
693 GLU   93  OE1
694 GLU   93  OE2
695 ASP   94  N
696 ASP   94  CA
697 ASP   94  C
701 ASP   94  OD1
703 LEU   95  N
704 LEU   95  CA
705 LEU   95  C
707 LEU   95  CB
711 LEU   96  N
712 LEU   96  CA
713 LEU   96  C
714 LEU   96  O
715 LEU   96  CB
716 LEU   96  CG
717 LEU   96  CD1
718 LEU   96  CD2
719 ASP   97  N
726 ASP   97  OD2
747 ARG   101  C
748 ARG   101  O
753 ARG   101  CZ
755 ARG   101  NH2
756 LYS   102  N
757 LYS   102  CA
758 LYS   102  C
760 LYS   102  CB
761 LYS   102  CG
762 LYS   102  CD
763 LYS   102  CE
765 PHE   103  N
766 PHE   103  CA
767 PHE   103  C
768 PHE   103  O
769 PHE   103  CB
770 PHE   103  CG
771 PHE   10

### **Second Method:**

Use a spatial data structure to store the atoms of one of the files. Then, for each atom in the other file, find all the atoms in the spatial

data structure that are closer than the distance threshold. If any are found, the two atoms overlap. More efficient, but more complex.

**`k-d tree`**

- Consists of a tree that is used to store k-dimensional points, in this case 3D points. For every of the non-leaf nodes, a 

  splitting hyperplane is generated dividing the space into two half-spaces. The hyperplane is defined based on one of the k-dimensions, such 

  that all points in the left subtree have a smaller coordinate value for that dimension than the node's point, and all points in the right 

  subtree have a larger coordinate value.

<br>

- This algorithm selects an axis to split the space into two subspaces, and then recursively builds a k-d tree on each of the subspaces. The

  axis selection is done by choosing the axis with the largest spread in the data. The splitting point is chosen as the median of the data

  along the selected axis.

<br>

- The searching algorithm is similar to the one used in binary search trees. The search starts at the root node, and then recursively 
  
  searches the left or right subtree depending on the splitting hyperplane. The search ends when the current node is a leaf node, or when the
  
  distance to the splitting hyperplane is larger than the distance to the closest point found so far. If the range intersects the splitting 
  
  hyperplane, the algorithm must search both subtrees. Otherwise, it only needs to search the subtree that contains the query point.
  
<br>

- The complexity of the algorithm is O(log n) for balanced trees, and O(n) for unbalanced trees compared to O(n^2) for the first method, which
  
  has to go through all the atoms in both files for each atom in one of the files.
  
<br>

- k-d trees are useful in this specific context because they allow for efficient searching of points in a 3D space. This is useful because

  it allows us to find all the atoms in a given range of a point in a 3D space, which is exactly what we need to do to find all the atoms that

  are closer than the distance threshold to a given atom. The number of comparisons is therefore reduced from O(n*m) to O(log n) for balanced

  trees, and O(n) for unbalanced trees.


In [18]:
from scipy.spatial import KDTree

def create_kdtree(atoms):
    coords = []
    for atom_number in atoms:
        coords.append(atoms[atom_number][1])
    return KDTree(coords)

def find_overlapping_atoms(kdtree, atoms, threshold=4.0):
    overlaps = []
    num_comp = 0
    
    for atom in atoms:
        points = kdtree.query_ball_point(atoms[atom][1], threshold)
        num_comp += len(points)
        
        if points:
            overlap = str(atoms[atom][0])
            overlaps.append(overlap)
    
    return overlaps, num_comp

kdtree_1 = create_kdtree(atoms2)

overlaps, num_comp = find_overlapping_atoms(kdtree_1, atoms1)

print("Number of comparisons: ", num_comp)
print("Number of overlaps: ", len(overlaps))
print("Overlaps:")
with open('overlaps2.txt', 'w') as file:
    file.write("Number of comparisons: " + str(num_comp) + '\n')
    file.write("Number of overlaps: " + str(len(overlaps)) + '\n')
    for overlap in overlaps:
        file.write(overlap + '\n')
        print(overlap)


Number of comparisons:  7265
Number of overlaps:  850
Overlaps:
674 SER   91  C
675 SER   91  O
678 LEU   92  N
679 LEU   92  CA
680 LEU   92  C
681 LEU   92  O
682 LEU   92  CB
683 LEU   92  CG
684 LEU   92  CD1
685 LEU   92  CD2
686 GLU   93  N
687 GLU   93  CA
688 GLU   93  C
689 GLU   93  O
690 GLU   93  CB
691 GLU   93  CG
692 GLU   93  CD
693 GLU   93  OE1
694 GLU   93  OE2
695 ASP   94  N
696 ASP   94  CA
697 ASP   94  C
701 ASP   94  OD1
703 LEU   95  N
704 LEU   95  CA
705 LEU   95  C
707 LEU   95  CB
711 LEU   96  N
712 LEU   96  CA
713 LEU   96  C
714 LEU   96  O
715 LEU   96  CB
716 LEU   96  CG
717 LEU   96  CD1
718 LEU   96  CD2
719 ASP   97  N
726 ASP   97  OD2
747 ARG   101  C
748 ARG   101  O
753 ARG   101  CZ
755 ARG   101  NH2
756 LYS   102  N
757 LYS   102  CA
758 LYS   102  C
760 LYS   102  CB
761 LYS   102  CG
762 LYS   102  CD
763 LYS   102  CE
765 PHE   103  N
766 PHE   103  CA
767 PHE   103  C
768 PHE   103  O
769 PHE   103  CB
770 PHE   103  CG
771 PHE   103  